##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introdução ao RL e às redes Deep Q

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/0_intro_rl">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/0_intro_rl.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/0_intro_rl.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/agents/tutorials/0_intro_rl.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Introdução

O aprendizado por reforço (RL) é um framework geral em que os agentes aprendem a realizar ações em um ambiente para maximizar uma recompensa. Os dois componentes principais são o ambiente, que representa o problema a ser resolvido, e o agente, que representa o algoritmo de aprendizado.

O agente e o ambiente interagem continuamente entre si. A cada timestep, o agente realiza uma ação no ambiente com base na sua *política* $\pi(a_t|s_t)$, em que $s_t$ é a observação atual do ambiente, e recebe uma recompensa $r_{t+1}$ e a próxima observação $s_{t+1}$ do ambiente. O objetivo é melhorar a política maximizando a soma das recompensas (retorno).

Observação: é importante distinguir entre o `state` (estado) do ambiente e a `observation` (observação), que faz parte do `state` do ambiente que o agente pode ver. Por exemplo, em um jogo de poker, o estado do ambiente consiste nas cartas que pertencem a todos os jogadores e às cartas comunitárias, mas o agente pode observar apenas suas próprias cartas e algumas cartas comunitárias. Na maior parte da literatura, esses termos são usados de forma intercambiável e a observação também é indicada como $s$.

![Loop de interação agente-ambiente](images/rl_overview.png)

Esse é um framework bastante geral e pode ser modelo para uma variedade de problemas de tomada de decisões sequenciais, como jogos, robótica etc.


## Ambiente Cartpole

O ambiente Cartpole é um dos problemas clássicos de aprendizado por reforço mais conhecidos ( o *"Olá, Mundo!"* do RL). Um pêndulo é acoplado a um carrinho, que pode se mover ao longo de um trilho sem atrito. O pêndulo começa na vertical e o objetivo é evitar que ele caia controlando o carrinho.

- A observação do ambiente $s_t$ é um vetor 4D que representa a posição e a velocidade do carrinho, além do ângulo e da velocidade angular do pêndulo.
- O agente pode controlar o sistema realizando uma das 2 ações $a_t$: empurrar o carrinho para a direita (+1) ou para a esquerda (-1).
- Uma recompensa $r_{t+1} = 1$ é fornecida a cada timestep que o pêndulo permanece na posição vertical. O episódio termina quando uma das seguintes condições for verdadeira:
    - o pêndulo se inclina ultrapassando um limite de ângulo
    - o carrinho se move para fora das bordas do mundo
    - passam 200 timesteps.

O objetivo do agente é aprender uma política $\pi(a_t|s_t)$ para maximizar a soma das recompensas em um episódio $\sum_{t=0}^{T} \gamma^t r_t$. Aqui, $\gamma$ é um fator de desconto em $[0, 1]$ que desconta as recompensas futuras relativas às recompensas imediatas. Esse parâmetro nos ajuda a deixar a política focada, para que se importe mais em obter recompensas rapidamente.


## Agente DQN

O [algoritmo DQN (Deep Q-Network)](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) foi desenvolvido pela DeepMind em 2015. Ele conseguiu resolver uma ampla gama de jogos do Atari (alguns de nível sobre-humano) ao combinar o aprendizado por reforço e as redes neurais profundas em larga escala. O algoritmo foi desenvolvido aprimorando um algoritmo de RL clássico chamado "Q-Learning", com redes neurais profundas e uma técnica chamada *replay de experiência*.

### Q-Learning

Q-Learning é baseado na noção de função Q. A função Q (também conhecida como função de valor estado-ação) de uma política $\pi$, $Q^{\pi}(s, a)$, mede o retorno esperado ou a soma descontada das recompensas obtidas do estado $s$ realizando <br> a ação $a$ primeiro e seguindo a política $\pi$ depois. Definimos a função Q ideal $Q^*(s, a)$ como o retorno máximo que pode ser obtido a partir da observação $s$, realizando a ação $a$ e seguindo a política ideal depois. A função Q ideal obedece à seguinte equação de otimalidade de *Bellman*:

```
$\begin{equation}Q^\ast(s, a) = \mathbb{E}[ r + \gamma \max_{a'} Q^\ast(s', a') ]\end{equation}$
```

Isso significa que o retorno máximo do estado $s$ e da ação $a$ é a soma da recompensa imediata $r$ e o retorno (descontado por $\gamma$) obtido seguindo a política ideal até o final do episódio (ou seja, a recompensa máxima do próximo estado $s'$). A expectativa é calculada tanto sobre a distribuição de recompensas imediatas $r$ quanto sobre os próximos estados possíveis $s'$.

A ideia básica por trás do Q-Learning é usar a equação de otimalidade de Bellman como uma atualização iterativa $Q_{i+1}(s, a) \leftarrow \mathbb{E}\left[ r + \gamma \max_{a'} Q_{i}(s', a')\right]$, e é possível mostrar que isso converge para a função $Q$ ideal, ou seja, $Q_i \rightarrow Q^*$ as $i \rightarrow \infty$ (veja o [artigo sobre a DQN](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf)).

### Deep Q-Learning

Para a maioria dos problemas, é impraticável representar a função $Q$ como uma tabela contendo valores para cada combinação de $s$ e $a$. Em vez disso, treinamos um aproximador de função, como uma rede neural com parâmetros $\theta$, para estimar os valores Q, ou seja, $Q(s, a; \theta) \approx Q^*(s, a)$. Isso pode ser feito minimizando a seguinte perda em cada etapa $i$:

$\begin{equation}L_i(\theta_i) = \mathbb{E}{em0}{s, a, r, s'\sim \rho(.)} \left[ (y_i - Q(s, a; \theta_i))^2 \right]\end{equation}$ where $y_i = r +  \gamma \max{/em0}{a'} Q(s', a'; \theta_{i-1})$

Aqui, $y_i$ é chamado de alvo TD (diferença temporal) e $y_i - Q$ é chamado de erro TD. $\rho$ representa a distribuição do comportamento e a distribuição sobre as transições ${s, a, r, s'}$ coletadas do ambiente.

Observe que os parâmetros da iteração anterior $\theta_{i-1}$ são fixos, e não atualizados. Na prática, usamos um instantâneo dos parâmetros de rede de algumas iterações atrás, em vez da última iteração. Essa cópia é chamada de *rede de destino*.

Q-Learning é um algoritmo *off-policy* que aprende sobre a política greedy $a = \max_{a} Q(s, a; \theta)$ usando uma política de comportamento diferente para agir no ambiente/coletar dados. Essa política de comportamento é geralmente uma política greedy $\epsilon$ que seleciona a ação greedy com probabilidade $1-\epsilon$ e uma ação aleatória com probabilidade $\epsilon$ para garantir uma boa cobertura do espaço estado-ação.

### Replay de experiência

Para evitar calcular a expectativa total da perda DQN, podemos minimizá-la usando o método do gradiente descendente estocástico. Se a perda for calculada usando apenas a última transição ${s, a, r, s'}$, isso se reduz ao Q-Learning padrão.

O trabalho da DQN no Atari introduziu uma técnica chamada "replay de experiência", para deixar as atualizações de rede mais estáveis. A cada timestep de coleta de dados, as transições são adicionadas a um buffer circular chamado *buffer de replay*. Durante o treinamento, em vez de usar apenas a última transição para calcular a perda e seu gradiente, usamos um minilote de transições amostradas do buffer de replay. Isso tem duas vantagens: melhor eficiência dos dados, ao reutilizar cada transição em várias atualizações, e melhor estabilidade, ao usar transições não correlacionadas em um lote.


## DQN no Cartpole com TF-Agents

O TF-Agents fornece todos os componentes necessários para treinar um agente DQN, como o próprio agente, o ambiente, as políticas, as redes, os buffers de replay, os loops de coleta de dados e as métricas. Esses componentes são implementados como funções Python ou operações de grafo do TensorFlow, e também temos wrappers para a conversão entre eles. Além disso, o TF-Agents oferece suporte ao modo TensorFlow 2.0, que permite o uso do TF no modo imperativo.

Em seguida, vamos conferir o [tutorial para treinar um agente DQN no ambiente Cartpole usando TF-Agents](https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb).
